In [2]:
from imports import *

In [3]:
from sklearn.metrics import precision_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import f1_score
import time

# Preprocessing
Load the dataset. IF is then run on samples of the dataset with different anomaly rates.

In [4]:
df = pd.read_csv('kddcup.data', delimiter=',')
df.columns = sa_columns + ["target"]

In [5]:
dfsf = df[df["logged_in"]== 1]
print(len(dfsf)- len(dfsf.loc[dfsf["target"]=='normal.']))
dfsf = dfsf[sf_columns + ["target"]] 
# print(dfsf.head(1))
# Split the dataset into 2 classes for consistent anomaly_rate when sampling
dfsf_normal = dfsf.loc[dfsf["target"]=='normal.']
dfsf_attack = dfsf.loc[dfsf["target"]!='normal.']
# print(f"A sample normal frame: \t {dfsf_normal.head(1)}")
# print("-----------------")
# print(f"A sample attack frame: \t {dfsf_attack.head(1)}")

3377


In [6]:
frac = 0.1

dfsf_frac = dfsf_normal.sample(frac = frac, random_state = 1).append(dfsf_attack.sample(frac = frac, random_state = 1))

anomaly_rate = 1.0 - len(dfsf_frac.loc[dfsf_frac["target"]=='normal.'])/len(dfsf_frac)
print(f"anomaly rate is {anomaly_rate:.1%} out of {len(dfsf_frac)} records")

dfsf_frac['binary_target'] = [1 if x=='normal.' else -1 for x in dfsf_frac["target"]]
toDecode = toDecodeSF
leSF = preprocessing.LabelEncoder()
for f in toDecode:
    dfsf_frac[f] = leSF.fit_transform(dfsf_frac[f])

dfsf_normed = preprocessing.normalize(dfsf_frac.drop(["target", "binary_target"], axis=1))

anomaly rate is 0.5% out of 70307 records


# Parameter tuning

In [29]:
def cross_validation_dbscan(eps, ms, silent=True):
    highest_score = 0
    best_eps = 0
    for e in eps:
        s = datetime.datetime.now()
        db = DBSCAN(eps=e, min_samples=ms, metric='euclidean', algorithm = 'auto', n_jobs=-1).fit(x_train)
        y_pred = [1 if i != -1 else i for i in db.labels_]
        t = datetime.datetime.now() - s
        p, r, f, s = map(lambda x: x[0], precision_recall_fscore_support(y_train, y_pred, labels=[-1]))
        # p = tp / (tp + fp)
        f = f1_score(y_train, y_pred, average = 'weighted')
        fa = f1_score(y_train, y_pred, pos_label = -1)
        p = precision_score(y_train, y_pred, pos_label = -1)
        r = recall_score(y_train, y_pred, pos_label = -1)
        a = roc_auc_score(y_train, y_pred)
        if r + fa > highest_score: highest_score = r + fa; best_eps = e
        
        yp = np.array(y_pred)
        yt = np.array(y_test)
        
        prediction = yp[np.where(yt == -1)]

        tp = len(yp[np.where(prediction == -1)])
        print(tp)
        # fp through formula
        fp = tp / p - tp
        
        # predicted true
        positives = yp[np.where(yp == -1)]
        if not silent:
            print(f"Finished trainning in {t} seconds")
            print(f"eps: {e} \t tp: {tp}/support: {s}/predicted: {len(positives)} -> fp = {len(positives) - tp}")
            print(f"AUC : {a:.1%} \t precision: {p:.3} \t recall: {r:.3} \t f1: {f} , {fa}")
            print("--------------------")
    return (best_eps, y_pred, f, len(positives) - tp, tp, p, r, t)

In [30]:
Y_axis_f1 = []
Y_recall = []
Y_precision = []
Y_axis_recall = []
best_param = []

In [31]:
rs = 1
x_train, x_test, y_train, y_test = train_test_split(dfsf_normed, dfsf_frac['binary_target'], test_size=0.25, random_state=rs)
stime = time.time()
emax, y_pred, f, fp, tp, p, r, t = cross_validation_dbscan(np.arange(0.0001, 0.01, 0.0005), 10, silent=False)
print("Time for DBSCAN fitting: %.3f" % (time.time() - stime))
Y_axis_f1.append(f)
Y_axis_recall.append(r)
Y_precision.append(p)
best_param.append(emax)

70
Finished trainning in 0:00:00.577804 seconds
eps: 0.0001 	 tp: 70/support: 241/predicted: 40364 -> fp = 40294
AUC : 22.0% 	 precision: 0.00124 	 recall: 0.207 	 f1: 0.3737481992738984 , 0.0024627508927471987
--------------------
34
Finished trainning in 0:00:00.844502 seconds
eps: 0.0006000000000000001 	 tp: 34/support: 241/predicted: 15934 -> fp = 15900
AUC : 44.8% 	 precision: 0.00301 	 recall: 0.199 	 f1: 0.8161935078307103 , 0.005935085007727975
--------------------
11
Finished trainning in 0:00:00.910827 seconds
eps: 0.0011 	 tp: 11/support: 241/predicted: 5972 -> fp = 5961
AUC : 52.4% 	 precision: 0.00653 	 recall: 0.162 	 f1: 0.9339539799128492 , 0.012554321583775953
--------------------
5
Finished trainning in 0:00:01.086133 seconds
eps: 0.0016 	 tp: 5/support: 241/predicted: 2916 -> fp = 2911
AUC : 55.1% 	 precision: 0.013 	 recall: 0.158 	 f1: 0.9655608003080132 , 0.024073487488121634
--------------------
3
Finished trainning in 0:00:01.127023 seconds
eps: 0.0021 	 tp: 3/s

In [ ]:
rs = 2
x_train, x_test, y_train, y_test = train_test_split(dfsf_normed, dfsf_frac['binary_target'], test_size=0.25, random_state=rs)
stime = time.time()
emax, y_pred, f, fp, tp, p, r, t = cross_validation_dbscan(np.arange(0.001, 0.01, 0.001), 15, silent=False)
print("Time for DBSCAN fitting: %.3f" % (time.time() - stime))
Y_axis_f1.append(f)
Y_axis_recall.append(r)
Y_precision.append(p)
best_param.append(emax)

18
Finished trainning in 0:00:00.922537 seconds
eps: 0.001 	 tp: 18/support: 252/predicted: 11117 -> fp = 11099
AUC : 49.4% 	 precision: 0.0045 	 recall: 0.198 	 f1: 0.8760683129405669 , 0.00879584835957428
--------------------
7
Finished trainning in 0:00:01.106119 seconds
eps: 0.002 	 tp: 7/support: 252/predicted: 2667 -> fp = 2660
AUC : 57.0% 	 precision: 0.018 	 recall: 0.19 	 f1: 0.9679792290900617 , 0.0328879753340185
--------------------
4
Finished trainning in 0:00:01.315582 seconds
eps: 0.003 	 tp: 4/support: 252/predicted: 1480 -> fp = 1476
AUC : 58.0% 	 precision: 0.0318 	 recall: 0.187 	 f1: 0.9797644759333587 , 0.054272517321016164
--------------------
2
Finished trainning in 0:00:01.411355 seconds
eps: 0.004 	 tp: 2/support: 252/predicted: 1039 -> fp = 1037
AUC : 58.2% 	 precision: 0.0443 	 recall: 0.183 	 f1: 0.9841063707759286 , 0.07126258714175057
--------------------
1
Finished trainning in 0:00:01.677904 seconds
eps: 0.005 	 tp: 1/support: 252/predicted: 808 -> fp = 

In [ ]:
rs = 2
x_train, x_test, y_train, y_test = train_test_split(dfsf_normed, dfsf_frac['binary_target'], test_size=0.25, random_state=rs)
stime = time.time()
emax, y_pred, f, fp, tp, p, r, t = cross_validation_dbscan(np.arange(0.001, 0.01, 0.001), 20, silent=False)
print("Time for DBSCAN fitting: %.3f" % (time.time() - stime))
Y_axis_f1.append(f)
Y_axis_recall.append(r)
Y_precision.append(p)
best_param.append(emax)

In [ ]:
rs = 4
x_train, x_test, y_train, y_test = train_test_split(dfsf_normed, dfsf_frac['binary_target'], test_size=0.25, random_state=rs)
stime = time.time()
emax, y_pred, f, fp, tp, p, r, t = cross_validation_dbscan(np.arange(0.001, 0.01, 0.001), 50, silent=False)
print("Time for DBSCAN fitting: %.3f" % (time.time() - stime))
Y_axis_f1.append(f)
Y_axis_recall.append(r)
Y_precision.append(p)
best_param.append(emax)

In [ ]:
# f1_tuned = [0.9954178136425095, 0.9952542140292294, 0.9953171367636211, 0.9952334249285871]

plt.figure(figsize=(40,20))
plt.rc('font', size=30)
plt.title('f1 score after parameter tuning')
plt.xlabel("configuration")
plt.ylabel("Score")
ax = plt.axes()
ax.set_ylim(0.994, 0.996)
pad = 0.005

print(best_param)

# labels = [f"{i}-{p}" for i,p in enumerate(best_param)]
labels = [f"{i}-{e:.2}" for i,e in enumerate(best_param)]
colors = {'1': 'r', '2':'b', '3':'c', '4':'y'}
scorer = 'F1'


# Y_axis = np.array([best_score_1, best_score_2, best_score_3, best_score_4])
# 
X_axis = np.array(labels)

ax.plot(X_axis, f1_tuned, '-', color=colors['1'],label="%s " % ("F1"), linewidth=5)
# ax.plot(X_axis, Y_axis_recall, '-', color=colors['2'],label="%s " % ("Recall"), linewidth=5)
plt.legend(loc="best")
plt.grid('off')
plt.tight_layout()
plt.savefig("DBSCAN_graph_v2.png", dpi=100)

plt.show()

# Comparing against default value

In [ ]:
f1_tuned = []
auc_tuned = []
r_tuned = []
f1_not_tuned = []
auc_not_tuned = []

In [ ]:
rs = 1
x_train, x_test, y_train, y_test = train_test_split(dfsf_normed, dfsf_frac['binary_target'], test_size=0.25, random_state=rs)

e = best_param[0]
ms = 10
print(e)

stime = time.time()
db = DBSCAN(eps=e, min_samples=ms, metric='euclidean', algorithm = 'auto', n_jobs=-1).fit(x_test)
y_pred = [1 if i != -1 else i for i in db.labels_]
print("Time for IF fitting: %.3f" % (time.time() - stime))
f = f1_score(y_test, y_pred)
print(f"---{f}")
f1_tuned.append(f)
a = roc_auc_score(y_test, y_pred)
print(f"---{a}")
auc_tuned.append(a)
r = recall_score(y_test, y_pred)
print(f"---{r}")
r_tuned.append(r)
db = DBSCAN(metric='euclidean', algorithm = 'auto', n_jobs=-1).fit(x_test)
y_pred = [1 if i != -1 else i for i in db.labels_]
a = roc_auc_score(y_test, y_pred)
print(f"---{a}")
auc_not_tuned.append(a)

In [ ]:
rs = 2
x_train, x_test, y_train, y_test = train_test_split(dfsf_normed, dfsf_frac['binary_target'], test_size=0.25, random_state=rs)

e = best_param[1]
ms = 15

stime = time.time()
db = DBSCAN(eps=e, min_samples=ms, metric='euclidean', algorithm = 'auto', n_jobs=-1).fit(x_test)
y_pred = [1 if i != -1 else i for i in db.labels_]
print("Time for IF fitting: %.3f" % (time.time() - stime))
f = f1_score(y_test, y_pred)
print(f"---{f}")
f1_tuned.append(f)
a = roc_auc_score(y_test, y_pred)
print(f"---{a}")
auc_tuned.append(a)
r = recall_score(y_test, y_pred)
print(f"---{r}")
r_tuned.append(r)
db = DBSCAN(metric='euclidean', algorithm = 'auto', n_jobs=-1).fit(x_test)
y_pred = [1 if i != -1 else i for i in db.labels_]
a = roc_auc_score(y_test, y_pred)
print(f"---{a}")
auc_not_tuned.append(a)

In [ ]:
rs = 3
x_train, x_test, y_train, y_test = train_test_split(dfsf_normed, dfsf_frac['binary_target'], test_size=0.25, random_state=rs)

e = best_param[2]
ms = 20

stime = time.time()
db = DBSCAN(eps=e, min_samples=ms, metric='euclidean', algorithm = 'auto', n_jobs=-1).fit(x_test)
y_pred = [1 if i != -1 else i for i in db.labels_]
print("Time for IF fitting: %.3f" % (time.time() - stime))
f = f1_score(y_test, y_pred)
print(f"---{f}")
f1_tuned.append(f)
a = roc_auc_score(y_test, y_pred)
print(f"---{a}")
auc_tuned.append(a)
r = recall_score(y_test, y_pred)
print(f"---{r}")
r_tuned.append(r)
db = DBSCAN(metric='euclidean', algorithm = 'auto', n_jobs=-1).fit(x_test)
y_pred = [1 if i != -1 else i for i in db.labels_]
a = roc_auc_score(y_test, y_pred)
print(f"---{a}")
auc_not_tuned.append(a)

In [ ]:
rs = 4
x_train, x_test, y_train, y_test = train_test_split(dfsf_normed, dfsf_frac['binary_target'], test_size=0.25, random_state=rs)

e = best_param[3]
ms = 50

stime = time.time()
db = DBSCAN(eps=e, min_samples=ms, metric='euclidean', algorithm = 'auto', n_jobs=-1).fit(x_test)
y_pred = [1 if i != -1 else i for i in db.labels_]
print("Time for IF fitting: %.3f" % (time.time() - stime))
f = f1_score(y_test, y_pred)
print(f"---{f}")
f1_tuned.append(f)
a = roc_auc_score(y_test, y_pred)
print(f"---{a}")
auc_tuned.append(a)
r = recall_score(y_test, y_pred)
print(f"---{r}")
r_tuned.append(r)
db = DBSCAN(metric='euclidean', algorithm = 'auto', n_jobs=-1).fit(x_test)
y_pred = [1 if i != -1 else i for i in db.labels_]
a = roc_auc_score(y_test, y_pred)
print(f"---{a}")
auc_not_tuned.append(a)

In [ ]:
fig, ax = plt.subplots(figsize = (30,20))
plt.rc('font', size=30)

f1_tuned = [0.9954178136425095, 0.9952542140292294, 0.9953171367636211, 0.9952334249285871]
f1_default = [0.9615918309257809, 0.955213461022107, 0.9576507857113085, 0.9571646010002174]
width = 1
labels = [f"eps: {best_param[0]}\ms: {10}", 
          f"eps: {best_param[0]}\ms: {15}", 
          f"eps: {best_param[0]}\ms: {20}", 
          f"eps: {best_param[0]}\ms: {50}"]
ax.set_xticks([i*5 + width/2 for i in range(len(f1_tuned))])
ax.set_xticklabels(labels)
ax.set_ylabel('auc-score')

ax.set_title('auc score before and after parameter tuning using cross validation search')

ax.bar([i*5 for i in range(len(auc_tuned))], auc_tuned, width=width)
ax.bar([i*5 + width for i in range(len(auc_not_tuned))], auc_not_tuned, width=width)
fig.tight_layout()
ax.legend(['auc score tuned', 'auc score not tuned'])
plt.show()

# Means and standard deviation

In [ ]:
import statistics 
time = [0.287, 0.298, 0.293, 0.366]
m = statistics.mean(time)
s = statistics.stdev(time)
print(f"time:\t {m} \pm {s}")
m = statistics.mean(auc_tuned)
s = statistics.stdev(auc_tuned)
print(f"auc:\t {m} \pm {s}")
m = statistics.mean(f1_tuned)
s = statistics.stdev(f1_tuned)
print(f"f1:\t {m} \pm {s}")
m = statistics.mean(r_tuned)
s = statistics.stdev(r_tuned)
print(f"recall:\t {m} \pm {s}")